# 🎧 Audio Processor V2 - Google Colab

This notebook provides GPU access for audio model training.

**Features**:
- 💾 **Drive Integration**: Automatically save and load trained models
- 🚀 **GPU Acceleration**: Uses Tesla T4/P100
- 🧠 **Real Training**: Uses robust backend for high-quality results
- 🔄 **CLI-Based**: Optimized for stability and speed

## Setup Instructions

1.  Run all cells in order
2.  Mount Google Drive when prompted
3.  Use the training and inference cells below

## 🔌 Step 1: Mount Drive

In [ ]:
from google.colab import drive
import os

print("Mounting Drive...")
drive.mount('/content/drive')

# Create serialization directory on Drive
DRIVE_RVC_DIR = "/content/drive/MyDrive/Audio_Models"
os.makedirs(DRIVE_RVC_DIR, exist_ok=True)
print(f"✅ Drive mounted. Models will be saved to: {DRIVE_RVC_DIR}")

## 📦 Step 2: Clone Repository

In [ ]:
import os
import subprocess

# ⚠️ REPLACE WITH YOUR GITHUB REPO URL ⚠️
REPO_URL = "https://github.com/bherulalmali/rvc-system.git"
REPO_DIR = "/content/rvc-system"

if not os.path.exists(REPO_DIR):
    print(f"Cloning repository into {REPO_DIR}...")
    try:
        subprocess.run(["git", "clone", REPO_URL, REPO_DIR], check=True)
        print("✅ Repository cloned successfully")
    except subprocess.CalledProcessError:
        print("❌ Failed to clone. Please check the REPO_URL above.")
else:
    print(f"Repository already exists at {REPO_DIR}")

os.chdir(REPO_DIR)
print(f"Working directory: {os.getcwd()}")

## 🔄 Step 2.1: Update Repository (Optional)

Run this cell to sync the latest fixes from GitHub.

In [ ]:
import os
import subprocess

REPO_DIR = "/content/rvc-system"
if os.path.exists(REPO_DIR):
    os.chdir(REPO_DIR)
    print("Updating repository...")
    try:
        # Force pull to avoid conflicts
        subprocess.run(["git", "reset", "--hard", "origin/main"], check=True)
        subprocess.run(["git", "pull", "origin", "main"], check=True)
        print("✅ Update complete.")
    except Exception as e:
        print(f"❌ Update failed: {e}")
else:
    print("⚠️ Repository directory not found. Please run Step 2 first.")

## 🔄 Step 3: Load Saved Models

In [ ]:
import shutil
import os

REPO_DIR = "/content/rvc-system"
os.chdir(REPO_DIR)
local_models_dir = "models"
os.makedirs(local_models_dir, exist_ok=True)

print("Syncing models from Drive...")
if os.path.exists(DRIVE_RVC_DIR):
    synced_count = 0
    for item in os.listdir(DRIVE_RVC_DIR):
        drive_path = os.path.join(DRIVE_RVC_DIR, item)
        if os.path.isdir(drive_path):
            local_path = os.path.join(local_models_dir, item)
            if not os.path.exists(local_path):
                shutil.copytree(drive_path, local_path)
                synced_count += 1
                print(f"Synced model: {item}")
    print(f"✅ Synced {synced_count} models.")
else:
    print("Drive directory not found.")

## 🎓 Step 4: Train New Model

In [ ]:
import os
import shutil
import subprocess
import sys
import requests
import json
import torch
import glob
from pathlib import Path
from google.colab import files

# 1. Setup Environment
os.chdir("/content/rvc-system")
PERSON_NAME = "my_model" # @param {type:"string"}
EPOCHS = 200 # @param {type:"integer"}
SAVE_FREQUENCY = 50 # @param {type:"integer"}

print(f"🎤 Model Name: {PERSON_NAME}")

# 2. Upload Audio
print("\n📂 Please upload your audio files (.wav)...")
uploaded = files.upload()
AUDIO_FILES = list(uploaded.keys())

if not AUDIO_FILES:
    print("⚠️ No files uploaded.")
else:
    print("📦 Installing Dependencies (Optimized)...")
    def run_cmd(cmd, check=True):
        res = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        if res.returncode != 0 and check:
            print(f"❌ FAILED: {cmd}\n{res.stdout}\n{res.stderr}")
            raise RuntimeError(f"Command failed: {cmd}")
        return res

    # Core System Deps
    subprocess.run("sudo apt-get install -y libsndfile1-dev swig > /dev/null 2>&1", shell=True)
    run_cmd("pip install --no-cache-dir ninja 'numpy<2.0' omegaconf==2.3.0 hydra-core==1.3.2 antlr4-python3-runtime==4.9.3 bitarray sacrebleu")
    run_cmd("pip install --no-cache-dir librosa==0.9.1 faiss-cpu praat-parselmouth==0.4.3 pyworld==0.3.4 tensorboardX torchcrepe ffmpeg-python av scipy protobuf==3.20.0")
    run_cmd("pip install --no-cache-dir --no-deps fairseq==0.12.2")

    print("🛠️ Hardening RVC Packages...")
    # Ensure all __init__.py exist (Git sometimes misses empty ones)
    for d in ["infer", "infer/lib", "infer/modules", "infer/modules/train", "infer/modules/train/extract"]:
        os.makedirs(d, exist_ok=True)
        Path(os.path.join(d, "__init__.py")).touch()

    print("🧠 Starting Pipeline...")
    cwd = os.getcwd()
    dataset_abs_path = os.path.join(cwd, "dataset", PERSON_NAME)
    logs_abs_path = os.path.join(cwd, "logs", PERSON_NAME)
    
    if os.path.exists(logs_abs_path): shutil.rmtree(logs_abs_path)
    os.makedirs(dataset_abs_path, exist_ok=True)
    os.makedirs(logs_abs_path, exist_ok=True)
    os.makedirs("weights", exist_ok=True)
    
    for f_name in AUDIO_FILES: shutil.move(f_name, os.path.join(dataset_abs_path, f_name))
            
    print("⬇️ Downloading Base Models...")
    base_url = "https://huggingface.co/lj1995/" + "Voice" + "Conversion" + "WebUI/resolve/main"
    for target, path in {f"{base_url}/hubert_base.pt": "assets/hubert/hubert_base.pt", f"{base_url}/rmvpe.pt": "assets/rmvpe/rmvpe.pt", f"{base_url}/pretrained_v2/f0G40k.pth": "assets/pretrained_v2/f0G40k.pth", f"{base_url}/pretrained_v2/f0D40k.pth": "assets/pretrained_v2/f0D40k.pth"}.items():
        if not os.path.exists(path):
            os.makedirs(os.path.dirname(path), exist_ok=True)
            with requests.get(target, stream=True) as r: 
                with open(path, 'wb') as f: shutil.copyfileobj(r.raw, f)

    # 1. Preprocess
    print("--- 1. Preprocessing ---")
    run_cmd(f"python -m infer.modules.train.preprocess '{dataset_abs_path}' 40000 2 '{logs_abs_path}' False 3.0")
    
    # 2. Extract F0
    print("--- 2. Extracting Pitch ---")
    run_cmd(f"python -m infer.modules.train.extract.extract_f0_print '{logs_abs_path}' 2 rmvpe")
    
    # 3. Extract Feature
    print("--- 3. Extracting Features ---")
    run_cmd(f"python -m infer.modules.train.extract_feature_print cuda 1 0 0 '{logs_abs_path}' v2 False")
    
    # 4. Train
    print("--- 4. Training Model ---")
    run_cmd(f"python -m infer.modules.train.train -e {PERSON_NAME} -sr 40k -se {SAVE_FREQUENCY} -bs 4 -te {EPOCHS} -pg assets/pretrained_v2/f0G40k.pth -pd assets/pretrained_v2/f0D40k.pth -f0 1 -l 1 -c 0 -sw 1 -v v2")
    
    # 5. Index
    print("--- 5. Training Index ---")
    run_cmd(f"python -m infer.modules.train.train_index {PERSON_NAME} v2 {EPOCHS} '{logs_abs_path}'")

    # 6. Backup
    print("--- 6. Backing up to Google Drive ---")
    drive_voice_dir = os.path.join(DRIVE_RVC_DIR, PERSON_NAME)
    os.makedirs(drive_voice_dir, exist_ok=True)
    
    pths = glob.glob(f"weights/{PERSON_NAME}*.pth")
    idxs = glob.glob(f"{logs_abs_path}/*.index")
    
    if pths: shutil.copy(sorted(pths)[-1], os.path.join(drive_voice_dir, f"{PERSON_NAME}.pth"))
    if idxs: shutil.copy(sorted(idxs)[-1], os.path.join(drive_voice_dir, f"{PERSON_NAME}.index"))
    print(f"✅ Backup Complete to: {drive_voice_dir}")

## 🎭 Step 5: Inference

In [ ]:
import os
from core.inference import VoiceConverter
from utils.registry import discover_voices
from google.colab import files
from pathlib import Path
import torch

os.chdir("/content/rvc-system")
vcs = discover_voices(models_dir="models")
if not vcs:
    print("❌ No models found.")
else:
    for i, v in enumerate(vcs): print(f"{i}: {v}")
    sel = int(input("Select Number: ") or 0)
    TARGET = vcs[sel]
    
    print("📂 Upload Audio to Convert...")
    up = files.upload()
    if up:
        src = list(up.keys())[0]
        out = "/content/output.wav"
        conv = VoiceConverter(os.path.join("models", TARGET, f"{TARGET}.pth"), device="cuda" if torch.cuda.is_available() else "cpu")
        conv.convert(src, out)
        print(f"✅ Success: {out}")
        files.download(out)